<a href="https://colab.research.google.com/github/AlexUrtubia/prediccion_ventas_comida/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning

El siguiente cuaderno contiene la secuencia de acciones necesarias para la limpieza de los datos del dataframe a utilizar para las predicciones

# Creando el dataframe 

En primer lugar es necesario crear el dataframe original, a partir de algún archivo externo que necesite ser cargado a google colab para trabajar sobre este

In [ ]:
# Al importar drive de google.colab, se concede acceso a nuestro contenido guardado en Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Se indica la ruta del archivo que se necesite utilizar como base del dataframe en una variable 
# y posteriormente (para este caso), se utiliza 'read_csv' de pandas para la lectura del archivo csv 
import pandas as pd # Por supuesto es necesario también importar pandas bajo algún alias que facilite su importación
filename = '/content/drive/MyDrive/Colab Notebooks/Data Science - Coding Dojo/Proyecto1/sales_predictions.csv'
df = pd.read_csv(filename)
print(df) # Se imprime el dataframe

     Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
0              FDA15        9.300          Low Fat         0.016047   
1              DRC01        5.920          Regular         0.019278   
2              FDN15       17.500          Low Fat         0.016760   
3              FDX07       19.200          Regular         0.000000   
4              NCD19        8.930          Low Fat         0.000000   
...              ...          ...              ...              ...   
8518           FDF22        6.865          Low Fat         0.056783   
8519           FDS36        8.380          Regular         0.046982   
8520           NCJ29       10.600          Low Fat         0.035186   
8521           FDN46        7.210          Regular         0.145221   
8522           DRG01       14.800          Low Fat         0.044878   

                  Item_Type  Item_MRP Outlet_Identifier  \
0                     Dairy  249.8092            OUT049   
1               Soft Drinks  

In [ ]:
# 'info' proporciona información importante acerca del df, tal como cantidad de elementos, filas, columnas, 
# sus nombres, el tipo de variable que guarda cada columna y la cantidad de datos no-nulos de cada una
df.info()
# Es posible observar que de un total de filas, dos de ellas contienen valores nulos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [ ]:
# Describe proporciona datos estadísticos del df, tales como media, desvíación estándar, cuartiles, mínimo, máximo, etc. de cada columna
df.describe()
# De manera preliminar, pareciera que no existen datos con valores atípicos, salvo por el mínimo de 'Item_Visibility', que tienen 0 como valor
# el cual debería revisarse con mayor detenimiento más adelante

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
# Se crea un respaldo del df original bajo el nombre de df1
df1 = df

In [ ]:
# Al revisar con value_counts los datos de la columna Item_Fat_Content, vemos que hay cierta inconcistencia en los datos
df1['Item_Fat_Content'].value_counts()
# Solo hay dos variedades, low fat o regular, pero están tipeados de distinta manera, por lo que para evitar errores futuros se busca mantener una uniformidad en las opciones

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [ ]:
# Se itera a traves de la columna con un for consultando con loc si es que el elemento de dicha fila y de la columna 'Item_Fat_Content'
# es distinto a 'Low Fat' o 'Regular', se actualiza con estos valores
for i in range (len(df1)):
  if df.loc[i, 'Item_Fat_Content'] == 'LF':
    df.loc[i, 'Item_Fat_Content'] = 'Low Fat'
  if df.loc[i, 'Item_Fat_Content'] == 'low fat':
    df.loc[i, 'Item_Fat_Content'] = 'Low Fat'
  if df.loc[i, 'Item_Fat_Content'] == 'reg':
    df.loc[i, 'Item_Fat_Content'] = 'Regular'

In [ ]:
# Se consulta nuevamente y se comprueba que ahora si hay uniformidad en los datos de esta columna
df1['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

# Comprobando datos duplicados

Comprobar la existencia de datos duplicados es una de las partes más importantes de la limpieza de los datos, ya que la existencia de estos, podría generar muchos errores hacia adelante.

In [ ]:
# El método duplicated retorna una serie de pandas con True o False indicando la posición de posibles datos falsos en cada fila
# Al añadir .sum se obtiene la totalidad de datos duplicados a lo largo de toda la serie
df1.duplicated().sum()

#Es posible observar que en este caso, no existen datos duplicados

0

# Revisando datos nulos

La parte más tediosa y compleja de una predicción siempre será la limpieza de los datos, ya sea eliminando una parte de ellos o rellenandolos de alguna manera, por lo que se vuelve fundamental comprender si es que existe alguna relación entre los datos faltantes 

# Item Weight

In [ ]:
# Una manera de revisar columna por columna la cantidad de datos nulos que contiene cada una, 
# es utilizando 'isnull' que retorna una copia del df original pero con valores booleanos que indican si el valor es un dato nulo o no
# Al añadir sum, se suma a cantidad de 'True' (o valores efectivamente nulos) contenidas en el dt creado 
df1.isnull().sum()
# Podemos observar que las columnas que contienen datos nulos son aquellas correspondientes al peso de los ítems y al tamaño de las tiendas

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
# Para comenzar a entender la naturaleza de estos datos nulos, se busca en primer lugar realizar un análisis a la columna Item_Weight
# Se crea una serie que contiene los datos nulos de dicha columna
null_weight = df1['Item_Weight'].isnull()
print(null_weight)

0       False
1       False
2       False
3       False
4       False
        ...  
8518    False
8519    False
8520    False
8521    False
8522    False
Name: Item_Weight, Length: 8523, dtype: bool


In [ ]:
# Esta serie se usa como filtro para el df, que contendrá solo las filas en donde existan valores nulos para la columna Item_Weight
# Se guarda en un nuevo df que contendrá a un extracto del original con los datos relevantes para este caso.
df_weight_null = df1[null_weight]
df_weight_null

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
18,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680
21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
...,...,...,...,...,...,...,...,...,...,...,...,...
8485,DRK37,NaN,Low Fat,0.043792,Soft Drinks,189.0530,OUT027,1985,Medium,Tier 3,Supermarket Type3,6261.8490
8487,DRG13,NaN,Low Fat,0.037006,Soft Drinks,164.7526,OUT027,1985,Medium,Tier 3,Supermarket Type3,4111.3150
8488,NCN14,NaN,Low Fat,0.091473,Others,184.6608,OUT027,1985,Medium,Tier 3,Supermarket Type3,2756.4120
8490,FDU44,NaN,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


Se busca alguna relación entre el peso del producto y otras columnas del dataframe

In [ ]:
# Se comprueba los distintos valores y su frecuencia para la columna Item_Type
df_weight_null['Item_Type'].value_counts()
# No pareciera tener alguna relación con el tipo de ítem

Fruits and Vegetables    213
Snack Foods              212
Household                151
Frozen Foods             138
Dairy                    116
Baking Goods             112
Canned                   110
Health and Hygiene        90
Meat                      88
Soft Drinks               71
Breads                    47
Others                    32
Hard Drinks               31
Breakfast                 21
Starchy Foods             18
Seafood                   13
Name: Item_Type, dtype: int64

In [ ]:
# Lo mismo para el contenido graso del producto
df_weight_null['Item_Fat_Content'].value_counts()
# Tampoco demuestra tener una relación directa

Low Fat    951
Regular    512
Name: Item_Fat_Content, dtype: int64

In [ ]:
# Se revisa el id de las tiendas donde se venden
df_weight_null['Outlet_Identifier'].value_counts()
# Vemos que hay solo dos: OUT027 y OUT019

OUT027    935
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [ ]:
# En el dataframe original, la columna Outlet_Identifier tiene 10 elemento distintas, es decir en total son 10 tiendas
df['Outlet_Identifier'].value_counts()
# Además, la totalidad de las filas que están relacionadas a estos dos outlets no contienen información sobre el peso de los ítems

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

Se podría decir que al encontrar una relación directa entre el peso del producto y la tienda que los vende, el problema está en que estas dos tiendas, por algún motivo no han otorgado esta información en sus bases de datos

In [ ]:
# Se aplica un conteo de valores a la columna Item_Identifier para comprobar que del total de 1463 productos con valores nulos en el peso del ítem, 
# hay 1142 id distintas, algunas se repiten hasta dos veces
df_weight_null['Item_Identifier'].value_counts()

FDK08    2
FDA08    2
FDV23    2
FDY56    2
FDI04    2
        ..
FDM44    1
FDZ48    1
FDK41    1
FDD57    1
NCN18    1
Name: Item_Identifier, Length: 1142, dtype: int64

In [ ]:
# Se filtra dentro del mismo dt que contiene los datos nulos del peso con algún ID de estos
df_weight_null[df_weight_null['Item_Identifier'] == 'FDV23']
# Para este caso, el producto de id FDV23 aparece una vez por tienda.

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3896,FDV23,NaN,Low Fat,0.185307,Breads,125.6046,OUT019,1985,Small,Tier 1,Grocery Store,622.5230
4516,FDV23,NaN,Low Fat,0.105324,Breads,125.7046,OUT027,1985,Medium,Tier 3,Supermarket Type3,3237.1196


In [ ]:
# Qué pasa si se busca alguno de los id de estos productos en el df original?
# Se filtra en df1 todas aquellas filas en las que el id del producto sea 'FDV23'
df1[df['Item_Identifier']=='FDV23']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1096,FDV23,11.0,Low Fat,0.106051,Breads,124.4046,OUT045,2002,NaN,Tier 2,Supermarket Type1,2241.0828
3896,FDV23,NaN,Low Fat,0.185307,Breads,125.6046,OUT019,1985,Small,Tier 1,Grocery Store,622.5230
4516,FDV23,NaN,Low Fat,0.105324,Breads,125.7046,OUT027,1985,Medium,Tier 3,Supermarket Type3,3237.1196
6045,FDV23,11.0,Low Fat,0.106268,Breads,123.2046,OUT018,2009,Medium,Tier 3,Supermarket Type2,3237.1196
8370,FDV23,11.0,Low Fat,0.105817,Breads,126.0046,OUT035,2004,Small,Tier 2,Supermarket Type1,871.5322


Es posible observar que en total hay para este caso 5 productos con el mismo ID, además que este producto se comercializa en 5 locales distintas pero solo en OUT019 y en OUT027 no se otorga la información sobre su peso.

Se considera para este caso, que al tener todos el mismo peso, una buena estrategía podría ser que se rellenen estos datos faltantes con los que ya existen para el mismo producto vendido en las otras tiendas

In [ ]:
# Se busca conocer el valor más frecuente para el peso del producto de id FDV23 con el método mode() que devuelve la moda de la columna
df1[df['Item_Identifier']=='FDV23']['Item_Weight'].mode()

0    11.0
dtype: float64

Tiene sentido rellenar los datos faltantes con la moda de cada columna, pero es necesario realizar esto con algún iterador ya que realizar este trabajo manualmente conllevaría mucho tiempo que sería mejor emplearlo en otra tarea



In [ ]:
# Al observar el df utilizado para la limpieza de los datos nulos sobre el peso de los ítems, los índices de cada fila corresponden a los que tenía originalmente en el primer df
df_weight_null

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
18,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680
21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
...,...,...,...,...,...,...,...,...,...,...,...,...
8485,DRK37,NaN,Low Fat,0.043792,Soft Drinks,189.0530,OUT027,1985,Medium,Tier 3,Supermarket Type3,6261.8490
8487,DRG13,NaN,Low Fat,0.037006,Soft Drinks,164.7526,OUT027,1985,Medium,Tier 3,Supermarket Type3,4111.3150
8488,NCN14,NaN,Low Fat,0.091473,Others,184.6608,OUT027,1985,Medium,Tier 3,Supermarket Type3,2756.4120
8490,FDU44,NaN,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


A partir de este índice se podría aplicar una relación con el dataframe original de manera que al iterar sobre este, cuando exista alguna coincidencia de índices se actualice el valor del peso del ítem

In [ ]:
# Por lo que se vuelve necesario en primera instancia resetear el índice del df con los datos nulos del peso, para realizar correctamente las iteraciones
# reset.index añade una nueva columna con un índice desde 0 hasta la última fila del df
df_weight_null = df_weight_null.reset_index()
# Se actualiza este df y se imprime
df_weight_null
# Ahora el índice va desde 0 hasta 1462 y además el índice que tenían anteriormente se conserva en una nueva columna que se añadió llamada 'index'

,index,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
1,18,DRI11,NaN,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680
2,21,FDW12,NaN,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
3,23,FDC37,NaN,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
4,29,FDC14,NaN,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,8485,DRK37,NaN,Low Fat,0.043792,Soft Drinks,189.0530,OUT027,1985,Medium,Tier 3,Supermarket Type3,6261.8490
1459,8487,DRG13,NaN,Low Fat,0.037006,Soft Drinks,164.7526,OUT027,1985,Medium,Tier 3,Supermarket Type3,4111.3150
1460,8488,NCN14,NaN,Low Fat,0.091473,Others,184.6608,OUT027,1985,Medium,Tier 3,Supermarket Type3,2756.4120
1461,8490,FDU44,NaN,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


El iterador funcionará recorriendo leng, es decir fila por fila de los datos sin información sobre el peso del ítem.
En una variable llamada moda, se aplicará un filtro sobre el df original (df1) en donde se busque la id del ítem que sea igual a la id del ítem de leng de cada iteración (o de cada fila, pues cada iteración es una fila distinta hacia abajo) y se aplicará mode() sobre cada columna resultante para luego actualizar el peso en los datos NaN

In [ ]:
for i in range (len(df_weight_null)): # Se recorre cada fila de df_weight_null
  moda = df1[df['Item_Identifier']==df_weight_null.loc[i,'Item_Identifier']]['Item_Weight'].mode() # Se busca la moda de cada columna con el id de item de cada iteración
  for j in range (len(moda)): # En ocaciones la moda es más de una por lo que se aplica una doble iteración para solucionar este posible inconveniente
    df_weight_null.iloc[i,2] = moda[0] # Con iloc, se busca la fila i en su columna 1 (o directamente el peso de cada ítem) y se rellena o actualiza con la moda
    
    # df_weight_null[df_weight_null['Item_Identifier'] == items_id[i]]['Item_Weight'].fillna(moda[0], inplace=True)
      # A value is trying to be set on a copy of a slice from a DataFrame.
      # Try using .loc[row_indexer,col_indexer] = value instead"""
    
     # Entre ensallo y error se intentó actualizar cada dato faltante utilizando 'inplace=True' siempre devolvía un error
     # Se optó por directamente llamar y renombrar cada columna con loc o iloc para evitar utilizar este parámetro

In [ ]:
df_weight_null
# Se imprime y se comprueba que da resultado

,index,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
1,18,DRI11,8.260,Low Fat,0.034238,Hard Drinks,113.2834,OUT027,1985,Medium,Tier 3,Supermarket Type3,2303.6680
2,21,FDW12,8.315,Regular,0.035400,Baking Goods,144.5444,OUT027,1985,Medium,Tier 3,Supermarket Type3,4064.0432
3,23,FDC37,15.500,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
4,29,FDC14,14.500,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1458,8485,DRK37,5.000,Low Fat,0.043792,Soft Drinks,189.0530,OUT027,1985,Medium,Tier 3,Supermarket Type3,6261.8490
1459,8487,DRG13,17.250,Low Fat,0.037006,Soft Drinks,164.7526,OUT027,1985,Medium,Tier 3,Supermarket Type3,4111.3150
1460,8488,NCN14,19.100,Low Fat,0.091473,Others,184.6608,OUT027,1985,Medium,Tier 3,Supermarket Type3,2756.4120
1461,8490,FDU44,12.150,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


In [ ]:
# Sin embargo al volver a consultar si existen datos nulos sobre este df que los contiene, vemos que aún quedan 4 datos nulos
df_weight_null.isnull().sum()

index                        0
Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [ ]:
# Se un filtro sobre el df con los datos nulos sobre el peso, esta vez con los que aún con el rellenado siguen siendo nulos para una observación más en detalle
filtro2 = df_weight_null['Item_Weight'].isnull()
# Se guarda en un nuevo dt y se imprime
df_weight_null_2 = df_weight_null[filtro2]
df_weight_null_2

,index,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
171,927,FDN52,NaN,Regular,0.130933,Frozen Foods,86.9198,OUT027,1985,Medium,Tier 3,Supermarket Type3,1569.9564
335,1922,FDK57,NaN,Low Fat,0.079904,Snack Foods,120.0440,OUT027,1985,Medium,Tier 3,Supermarket Type3,4434.2280
687,4187,FDE52,NaN,Regular,0.029742,Dairy,88.9514,OUT027,1985,Medium,Tier 3,Supermarket Type3,3453.5046
819,5022,FDQ60,NaN,Regular,0.191501,Baking Goods,121.2098,OUT019,1985,Small,Tier 1,Grocery Store,120.5098


In [ ]:
# Al consultar en el df original sobre los demás productos con la misma ID, vemos que no hay otros aparte de sí mismos,
# Es decir, no existen otros datos de donde calcular la moda para poder rellenarlo
df1[df['Item_Identifier']==df_weight_null_2.iloc[2,0]]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


Se entiende que no hay otra manera de completar la información faltante para este caso, y al tratarse solamente de 4 filas del total de 8523, se opta por descartarlos y borrarlos del registro.

Este paso se hará al final de la limpieza

Una vez que se han eliminado o rellenado las filas sin información sobre el peso, es necesario traspasar esta infonrmación actualizada al df original (df1)

In [ ]:
# Se itera a travéz de la columna index del df con los datos nulos sobre el peso
# O dicho de otro modo, se recorre cada fila pero i tendrá el valor del índice correspondiente al df original
for i in df_weight_null['index']:
    df1.iloc[i,1] = df_weight_null[df_weight_null['index'] == i]['Item_Weight']
    # Se indica que en cada fila de df1, se actualice el peso del ítem con el valor de Item_Weight del df df_weight_null en donde coincidan los índices:
    # El índice original contenido en df1 con el valor de la columna 'index' de cada fila de df_weight_null

In [ ]:
# Se comprueba que esta vez, la columna Item_Weight del df original contiene solo los 4 valores nulos ya vistos
df1.isnull().sum()

Item_Identifier                 0
Item_Weight                     4
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

# Limpieza NaN Outlet Size

Al igual que en el caso anterior, se busca algua relación entre los datos faltantes con el resto del dataframe

In [ ]:
# En primer lugar se guarda un filtro que contenga los valores nulos de la columna Outlet_Size
null_osize = df1['Outlet_Size'].isnull()

In [ ]:
# Y se aplica sobre el df original
df_null_osize = df1[null_osize]
df_null_osize

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350
25,NCD06,13.000,Low Fat,0.099887,Household,45.9060,OUT017,2007,NaN,Tier 2,Supermarket Type1,838.9080
28,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,NaN,Tier 3,Grocery Store,178.4344
...,...,...,...,...,...,...,...,...,...,...,...,...
8502,NCH43,8.420,Low Fat,0.070712,Household,216.4192,OUT045,2002,NaN,Tier 2,Supermarket Type1,3020.0688
8508,FDW31,11.350,Regular,0.043246,Fruits and Vegetables,199.4742,OUT045,2002,NaN,Tier 2,Supermarket Type1,2587.9646
8509,FDG45,8.100,Low Fat,0.214306,Fruits and Vegetables,213.9902,OUT010,1998,NaN,Tier 3,Grocery Store,424.7804
8514,FDA01,15.000,Regular,0.054489,Canned,57.5904,OUT045,2002,NaN,Tier 2,Supermarket Type1,468.7232


In [ ]:
# Al consultar por los índices de los outlet de este df, vemos que solamente tres de ellos son los que no tienen los valores del tamaño del outlet
df_null_osize['Outlet_Identifier'].value_counts()

OUT045    929
OUT017    926
OUT010    555
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Tres de un total de 10 outlets, por algún motivo no informaron su tamaño
df['Outlet_Identifier'].value_counts()

OUT027    935
OUT013    932
OUT049    930
OUT046    930
OUT035    930
OUT045    929
OUT018    928
OUT017    926
OUT010    555
OUT019    528
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Consultando en el df base, vemos que existen tres tipos de Tier siendo Tier 3 el que más valores contiene
df1['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [ ]:
# Existen 4 tipos de outlet en el df original
df1['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [ ]:
# Si agrupasemos los datos por tipo de outlet y tipo de ubicación, podemos ver la cantidad que existe por cada uno
df1.groupby(['Outlet_Type'])['Outlet_Location_Type'].value_counts()
# Vemos que para el tipo 'Grocery Store' hay dos tipos de Tier, 1 y 3

Outlet_Type        Outlet_Location_Type
Grocery Store      Tier 3                   555
                   Tier 1                   528
Supermarket Type1  Tier 2                  2785
                   Tier 1                  1860
                   Tier 3                   932
Supermarket Type2  Tier 3                   928
Supermarket Type3  Tier 3                   935
Name: Outlet_Location_Type, dtype: int64

In [ ]:
# Al consular en el df original para outlet size, vemos que si agrupamos por tipo de ubicación y el tipo de outlet
df1.groupby(['Outlet_Location_Type', 'Outlet_Type'])['Outlet_Identifier'].value_counts()
# Vemos que hay 3 locales de Tier1 1, 3 de Tier 2 y 4 de Tier 3

Outlet_Location_Type  Outlet_Type        Outlet_Identifier
Tier 1                Grocery Store      OUT019               528
                      Supermarket Type1  OUT046               930
                                         OUT049               930
Tier 2                Supermarket Type1  OUT035               930
                                         OUT045               929
                                         OUT017               926
Tier 3                Grocery Store      OUT010               555
                      Supermarket Type1  OUT013               932
                      Supermarket Type2  OUT018               928
                      Supermarket Type3  OUT027               935
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Si consultamos lo mismo pero para el df df_null_osize, vemos que de los tres valores, 2 son de Tier 2 y además son Supermarket Type1, Out10 es un Grocery Store de tipo Tier3
df_null_osize.groupby(['Outlet_Location_Type', 'Outlet_Type'])['Outlet_Identifier'].value_counts()

Outlet_Location_Type  Outlet_Type        Outlet_Identifier
Tier 2                Supermarket Type1  OUT045               929
                                         OUT017               926
Tier 3                Grocery Store      OUT010               555
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Se aplica un filtro al df original en donde solo aparezcan los outlets de tipo 'Grocery Store'
gst = df1['Outlet_Type'] == 'Grocery Store'
df1[gst]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
23,FDC37,15.500,Low Fat,0.057557,Baking Goods,107.6938,OUT019,1985,Small,Tier 1,Grocery Store,214.3876
28,FDE51,5.925,Regular,0.161467,Dairy,45.5086,OUT010,1998,NaN,Tier 3,Grocery Store,178.4344
29,FDC14,14.500,Regular,0.072222,Canned,43.6454,OUT019,1985,Small,Tier 1,Grocery Store,125.8362
30,FDV38,19.250,Low Fat,0.170349,Dairy,55.7956,OUT010,1998,NaN,Tier 3,Grocery Store,163.7868
...,...,...,...,...,...,...,...,...,...,...,...,...
8473,DRI47,14.700,Low Fat,0.035016,Hard Drinks,144.3128,OUT010,1998,NaN,Tier 3,Grocery Store,431.4384
8480,FDQ58,7.315,Low Fat,0.000000,Snack Foods,154.5340,OUT019,1985,Small,Tier 1,Grocery Store,459.4020
8486,FDR20,20.000,Regular,0.000000,Fruits and Vegetables,46.4744,OUT010,1998,NaN,Tier 3,Grocery Store,45.2744
8490,FDU44,12.150,Regular,0.102296,Fruits and Vegetables,162.3552,OUT019,1985,Small,Tier 1,Grocery Store,487.3656


In [ ]:
# Si se compara bajo este filtro, cuales son los tamaños de las tiendas tipo Grocery Store, vemos que todas ellas son Small
df1[gst]['Outlet_Size'].value_counts()

Small    528
Name: Outlet_Size, dtype: int64

Esto podría llevar a concluir que el tamaño de OUT10 probablemente sea Small

In [ ]:
# Para añadir más información a esta posible conclusión, se filtran sobre el df original en todos los outlets de tamaño small, el tipo de outlet que son.
sm_out = df1['Outlet_Size'] == 'Small'
df1[sm_out]['Outlet_Type'].value_counts()
# Las opciones son 'Supermarket Type1' o 'Grocery Store'

Supermarket Type1    1860
Grocery Store         528
Name: Outlet_Type, dtype: int64

In [ ]:
# Agrupando por tipo de ubicación y tamaño de outlet, vemos que para los de tamaño pequeño solo hay en Tier 1 y Tier2, no Tier3 como es OUT10
df1.groupby(['Outlet_Location_Type', 'Outlet_Size'])['Outlet_Type'].value_counts()

Outlet_Location_Type  Outlet_Size  Outlet_Type      
Tier 1                Medium       Supermarket Type1    930
                      Small        Supermarket Type1    930
                                   Grocery Store        528
Tier 2                Small        Supermarket Type1    930
Tier 3                High         Supermarket Type1    932
                      Medium       Supermarket Type3    935
                                   Supermarket Type2    928
Name: Outlet_Type, dtype: int64

In [ ]:
# Al consultar en el df original la cantidad que vende cada tipo de outlet de vemos que las de tipo Grocery Store tienen en promedio casi 340
df1.groupby(['Outlet_Type'])['Item_Outlet_Sales'].mean()

Outlet_Type
Grocery Store         339.828500
Supermarket Type1    2316.181148
Supermarket Type2    1995.498739
Supermarket Type3    3694.038558
Name: Item_Outlet_Sales, dtype: float64

In [ ]:
# La misma cantidad que vende en promedio OUT10
df1.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].mean()

Outlet_Identifier
OUT010     339.351662
OUT013    2298.995256
OUT017    2340.675263
OUT018    1995.498739
OUT019     340.329723
OUT027    3694.038558
OUT035    2438.841866
OUT045    2192.384798
OUT046    2277.844267
OUT049    2348.354635
Name: Item_Outlet_Sales, dtype: float64

Por este motivo, se decide rellenar los datos sore el tamaño de Out10 con 'Small'.

Solo queda determinar los tamaños de OUT17 y OUT45

In [ ]:
# Agrupamos por id de tienda y consultamos el promedio de ventas 
df_null_osize.groupby(['Outlet_Identifier'])['Item_Outlet_Sales'].mean()

Outlet_Identifier
OUT010     339.351662
OUT017    2340.675263
OUT045    2192.384798
Name: Item_Outlet_Sales, dtype: float64

In [ ]:
# Se consulta el promedio de ventas por tamaño de outlet
df1.groupby(['Outlet_Size'])['Item_Outlet_Sales'].mean()
# Necesitamos profundizar respecto al promedio de ventas de las tiendas pequeñas

Outlet_Size
High      2298.995256
Medium    2681.603542
Small     1912.149161
Name: Item_Outlet_Sales, dtype: float64

In [ ]:
# Sabemos que las de tipo 'Grocery Store' reducen el promedio ya que venden menos pues tienen menos tipos de productos disponibles
df1.groupby(['Outlet_Type'])['Outlet_Identifier'].value_counts()

Outlet_Type        Outlet_Identifier
Grocery Store      OUT010               555
                   OUT019               528
Supermarket Type1  OUT013               932
                   OUT035               930
                   OUT046               930
                   OUT049               930
                   OUT045               929
                   OUT017               926
Supermarket Type2  OUT018               928
Supermarket Type3  OUT027               935
Name: Outlet_Identifier, dtype: int64

In [ ]:
# Por lo que se busca calcular el promedio de ventas según tamaño del outlet sin considerar las tiendas tipo 'Grocery Store'
gs = df1['Outlet_Type'] == 'Grocery Store'
df1[~gs].groupby(['Outlet_Size'])['Item_Outlet_Sales'].mean()

Outlet_Size
High      2298.995256
Medium    2681.603542
Small     2358.343066
Name: Item_Outlet_Sales, dtype: float64

Comparando los promedios de ventas de OUT017 y OUT045, tenemos 2340.675263 y 2192.384798 respecetivamente, similares a las de las tiendas pequeñas

In [ ]:
# Considerando además que para las tiendas con ubicación tipo Tier2, solo existen Supermarket Type1, se considera que OUT017 y OUT045, también se definirán como Small
df1.groupby(['Outlet_Location_Type', 'Outlet_Identifier'])['Outlet_Type'].value_counts()

Outlet_Location_Type  Outlet_Identifier  Outlet_Type      
Tier 1                OUT019             Grocery Store        528
                      OUT046             Supermarket Type1    930
                      OUT049             Supermarket Type1    930
Tier 2                OUT017             Supermarket Type1    926
                      OUT035             Supermarket Type1    930
                      OUT045             Supermarket Type1    929
Tier 3                OUT010             Grocery Store        555
                      OUT013             Supermarket Type1    932
                      OUT018             Supermarket Type2    928
                      OUT027             Supermarket Type3    935
Name: Outlet_Type, dtype: int64

Esta conclusón podría no ser la más acertada, puede ser que sean de tamaño Medium e incluso High, pero considerando que entre OUT017 y OUT045 hay 1855 datos, cerca del 22% del total, eliminar toda esta cantidad de datos, en donde por cierto se perdería por completo la información relacionada a dos tiendas.
 
Tiene más sentido tener un dato erroneo en una variable como esta que perder una cantidad de datos tan grande.

In [ ]:
# Por lo tanto se reemplazan todos los datos NaN de la columna 'Outlet_Size' por 'Small'
df1['Outlet_Size'] = df1['Outlet_Size'].fillna('Small')

In [ ]:
# Se comprueba nuevamente la cantidad de elementos nulos
df1.isnull().sum()

Item_Identifier              0
Item_Weight                  4
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

Vemos que solamente quedan los 4 datos nulos del peso de los ítem que no tenían otras repeticiones.

In [ ]:
# Al tratarse de 4 datos, simplemente se eliminan del df estos registros con dropna e inplace=True
df1.dropna(inplace=True)

In [ ]:
# De este modo vemos que el dataset ahora no contiene ningún dato nulo
df1.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

Revisando el df, más específicamente mediante el método describe, vemos que hay ciertos ítems de la columna Item_Visibility que tienen valor 0.

Al revisar la información sobre el df, vemos que esta columna define:	El porcentaje de área total de visualización de todos los productos en la tienda asignados a este producto particular.

Es decir, si hay productos con valor 0 indican que no están presentes en la tienda o porque simplemente no se les asignó el valor correcto.

In [ ]:
df1.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,8519.000000,8519.000000,8519.000000,8519.000000,8519.000000
mean,12.875420,0.066112,141.010019,1997.837892,2181.188779
std,4.646098,0.051586,62.283594,8.369105,1706.511093
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.785000,0.026983,93.844900,1987.000000,834.247400
50%,12.650000,0.053925,143.047000,1999.000000,1794.331000
75%,16.850000,0.094558,185.676600,2004.000000,3100.630600
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
# Se filtra en el df original ya limpio, aquellos productos con valor 0 en Item_Visibility.
it_v0 = df1['Item_Visibility'] == 0
df2 = df1[it_v0]
df2

# Vemos que en total, son 526 filas.

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
3,FDX07,19.200,Regular,0.0,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
10,FDY07,11.800,Low Fat,0.0,Fruits and Vegetables,45.5402,OUT049,1999,Medium,Tier 1,Supermarket Type1,1516.0266
32,FDP33,18.700,Low Fat,0.0,Snack Foods,256.6672,OUT018,2009,Medium,Tier 3,Supermarket Type2,3068.0064
...,...,...,...,...,...,...,...,...,...,...,...,...
8480,FDQ58,7.315,Low Fat,0.0,Snack Foods,154.5340,OUT019,1985,Small,Tier 1,Grocery Store,459.4020
8484,DRJ49,6.865,Low Fat,0.0,Soft Drinks,129.9652,OUT013,1987,High,Tier 3,Supermarket Type1,2324.9736
8486,FDR20,20.000,Regular,0.0,Fruits and Vegetables,46.4744,OUT010,1998,Small,Tier 3,Grocery Store,45.2744
8494,NCI54,15.200,Low Fat,0.0,Household,110.4912,OUT017,2007,Small,Tier 2,Supermarket Type1,1637.8680


In [ ]:
# De este df filtrado con los valores en 0, se consulta la cantidad de tiendas que tienen este tipo de productos.
df2.Outlet_Identifier.value_counts()
# Las 10 tiendas tienen este tipo de valores

OUT018    65
OUT046    61
OUT027    60
OUT013    59
OUT045    58
OUT017    57
OUT035    54
OUT049    53
OUT019    30
OUT010    29
Name: Outlet_Identifier, dtype: int64

Se comienza a buscar alguna relación entre estos valores y el resto de las columnas. 


In [ ]:
# Consultanto por el tipo de producto no se encuentra alguna relación, todos los productos tienen este dato
df2.Item_Type.value_counts()

Fruits and Vegetables    87
Snack Foods              68
Household                51
Frozen Foods             51
Baking Goods             45
Dairy                    38
Soft Drinks              34
Health and Hygiene       33
Canned                   33
Meat                     20
Breads                   17
Hard Drinks              17
Starchy Foods            15
Others                   10
Seafood                   4
Breakfast                 3
Name: Item_Type, dtype: int64

In [ ]:
# Para consultar por tienda, se crea un df filtrado en el que solamente aparezcan los productos de la tienda OUT018
df3 = df1[df1['Outlet_Identifier'] == 'OUT018']
df3

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
16,NCB42,11.800,Low Fat,0.008596,Health and Hygiene,115.3492,OUT018,2009,Medium,Tier 3,Supermarket Type2,1621.8888
31,NCS17,18.600,Low Fat,0.080829,Health and Hygiene,96.4436,OUT018,2009,Medium,Tier 3,Supermarket Type2,2741.7644
32,FDP33,18.700,Low Fat,0.000000,Snack Foods,256.6672,OUT018,2009,Medium,Tier 3,Supermarket Type2,3068.0064
...,...,...,...,...,...,...,...,...,...,...,...,...
8506,DRF37,17.250,Low Fat,0.084676,Soft Drinks,263.1910,OUT018,2009,Medium,Tier 3,Supermarket Type2,3944.8650
8511,FDF05,17.500,Low Fat,0.026980,Frozen Foods,262.5910,OUT018,2009,Medium,Tier 3,Supermarket Type2,4207.8560
8515,FDH24,20.700,Low Fat,0.021518,Baking Goods,157.5288,OUT018,2009,Medium,Tier 3,Supermarket Type2,1571.2880
8516,NCJ19,18.600,Low Fat,0.118661,Others,58.7588,OUT018,2009,Medium,Tier 3,Supermarket Type2,858.8820


In [ ]:
# Y ahora más específicamente se consulta por los items con visibilidad 0 en este df
df3[df3['Item_Visibility'] == 0]

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
5,FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
32,FDP33,18.700,Low Fat,0.0,Snack Foods,256.6672,OUT018,2009,Medium,Tier 3,Supermarket Type2,3068.0064
60,FDM20,10.000,Low Fat,0.0,Fruits and Vegetables,246.9144,OUT018,2009,Medium,Tier 3,Supermarket Type2,3185.1872
156,FDB36,5.465,Regular,0.0,Baking Goods,132.5626,OUT018,2009,Medium,Tier 3,Supermarket Type2,262.3252
307,FDR04,7.075,Low Fat,0.0,Frozen Foods,98.0068,OUT018,2009,Medium,Tier 3,Supermarket Type2,874.8612
...,...,...,...,...,...,...,...,...,...,...,...,...
7897,NCE54,20.700,Low Fat,0.0,Household,74.3354,OUT018,2009,Medium,Tier 3,Supermarket Type2,1880.8850
7952,DRF60,10.800,Low Fat,0.0,Soft Drinks,238.1564,OUT018,2009,Medium,Tier 3,Supermarket Type2,5243.8408
8253,DRG23,8.880,Low Fat,0.0,Hard Drinks,154.0682,OUT018,2009,Medium,Tier 3,Supermarket Type2,2896.8958
8323,FDH58,12.300,Low Fat,0.0,Snack Foods,116.4834,OUT018,2009,Medium,Tier 3,Supermarket Type2,1842.9344


In [ ]:
# Existirá quizás alguna relación entre la visibilidad y la cantidad de items vendidos?
df3[df3['Item_Visibility'] == 0].sort_values(by='Item_Outlet_Sales')
# No pareciera existir una relación

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
156,FDB36,5.465,Regular,0.0,Baking Goods,132.5626,OUT018,2009,Medium,Tier 3,Supermarket Type2,262.3252
6277,FDO60,20.000,Low Fat,0.0,Baking Goods,43.4086,OUT018,2009,Medium,Tier 3,Supermarket Type2,401.4774
505,FDK24,9.195,Low Fat,0.0,Baking Goods,43.6744,OUT018,2009,Medium,Tier 3,Supermarket Type2,407.4696
596,FDU04,7.930,Low Fat,0.0,Frozen Foods,123.2414,OUT018,2009,Medium,Tier 3,Supermarket Type2,487.3656
5760,FDC14,14.500,Regular,0.0,Canned,41.0454,OUT018,2009,Medium,Tier 3,Supermarket Type2,545.2902
...,...,...,...,...,...,...,...,...,...,...,...,...
5388,FDK52,18.250,Low Fat,0.0,Frozen Foods,224.1062,OUT018,2009,Medium,Tier 3,Supermarket Type2,4965.5364
7952,DRF60,10.800,Low Fat,0.0,Soft Drinks,238.1564,OUT018,2009,Medium,Tier 3,Supermarket Type2,5243.8408
423,FDA27,20.350,Regular,0.0,Dairy,256.7672,OUT018,2009,Medium,Tier 3,Supermarket Type2,5624.6784
1508,NCR53,12.150,Low Fat,0.0,Health and Hygiene,224.4404,OUT018,2009,Medium,Tier 3,Supermarket Type2,5626.0100


In [ ]:
# veamos ahora para esta tienda, la misma relación pero para aquellos productos que tienen un valor distinto de 0
df3[df3['Item_Visibility'] != 0].sort_values(by='Item_Outlet_Sales')
# No pareciera tampoco haber una relación, los productos más vendidos no tienen una visibilidad alta, ni viceversa, por lo que se descarta esta relación

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
5573,NCE31,7.670,Low Fat,0.185597,Household,35.7216,OUT018,2009,Medium,Tier 3,Supermarket Type2,69.2432
1657,FDL52,6.635,Regular,0.046278,Frozen Foods,36.7506,OUT018,2009,Medium,Tier 3,Supermarket Type2,75.9012
1543,FDA39,6.320,Low Fat,0.012770,Meat,39.9822,OUT018,2009,Medium,Tier 3,Supermarket Type2,78.5644
7318,FDV16,7.750,Regular,0.083269,Frozen Foods,32.9558,OUT018,2009,Medium,Tier 3,Supermarket Type2,101.8674
8103,FDT02,12.600,Low Fat,0.024293,Dairy,33.4874,OUT018,2009,Medium,Tier 3,Supermarket Type2,105.8622
...,...,...,...,...,...,...,...,...,...,...,...,...
2887,FDR25,17.000,Regular,0.140090,Canned,265.1884,OUT018,2009,Medium,Tier 3,Supermarket Type2,6359.7216
2282,NCX30,16.700,Low Fat,0.026729,Household,248.4776,OUT018,2009,Medium,Tier 3,Supermarket Type2,6439.6176
641,FDY51,12.500,Low Fat,0.081465,Meat,220.7798,OUT018,2009,Medium,Tier 3,Supermarket Type2,6611.3940
2803,FDU51,20.200,Regular,0.096907,Meat,175.5028,OUT018,2009,Medium,Tier 3,Supermarket Type2,6729.9064


In [ ]:
# Y entre los productos que si tienen visibilidad cuanto suman de manera acumulativa en la visibilidad?
df3[df3['Item_Visibility'] != 0]['Item_Visibility'].cumsum()
# De un total de 863 productos, solo suman el 56.6% del total de productos, lo cual no tiene mucho sentido, 
# pues solo restan 65 productos para completar el 43.4% restante de la visibilidad de los productos

1        0.019278
16       0.027874
31       0.108704
37       0.221828
43       0.290930
          ...    
8506    56.309074
8511    56.336054
8515    56.357572
8516    56.476234
8521    56.621454
Name: Item_Visibility, Length: 863, dtype: float64

Se realiza esta misma operación en cada tienda, y los resultados son similares, es decir, no existe una relación aparente entre estos datos con valor 0 y el resto de las columnas del df.

¿Qué podría significar este valor 0?
¿Realmente 0 está correcto como valor?

Es posible que los productos de cada tienda solo sean una muestra y no el total de cada una, y por este motivo la suma acumulativa de la visibilidad de los productos no cuadra con el resto de datos que tienen valor 0.

Es posible también que estos productos estén descontinuados, se vendan por internet o de alguna manera se comercializan/comercializaron sin estar visibles para el público.

Al no haber mayor información al respecto, se decide dejar los datos con el valor igual a 0, ya que no hay alguna manera de generar alguna relación entre estos valores con el resto de los datos disponibles.

Sin embargo, se creará también otro df en el que no hay valores 0 para la visibilidad, ya que son 526 productos de un total de 8523, es decir un poco más del 6% de los datos, lo que tampoco es un valor considerable.

In [ ]:
#Anteriormente se creo un filtro para aquellos valores con 0 visibility, se reutiliza para generar un filtro que permitirá excluir estos datos del df original
df_sin_v0 = df1[~it_v0]
df_sin_v0

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,19.000,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,Small,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [ ]:
# Se exporta el df como csv para continuar trabajando en él más adelante
df_sin_v0.to_csv(path_or_buf='df_sin_v0.csv',index = False)

In [ ]:
# Y también el df con los items con visibilidad 0
df1.to_csv(path_or_buf='df_v0.csv',index = False)